In [84]:
import numpy as np
import torch
import torch.nn as tnn
import torch.nn.functional as F
import torch.optim as topti
from torchtext import data
from torchtext.vocab import GloVe
from imdb_dataloader import IMDB

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device('cpu')

# 测试

In [22]:
textField = data.Field(lower=True, include_lengths=True, batch_first=True)
labelField = data.Field(sequential=False)

from imdb_dataloader import IMDB
train, dev = IMDB.splits(textField, labelField, train="train", validation="dev")

textField.build_vocab(train, dev, vectors=GloVe(name="6B", dim=50))
labelField.build_vocab(train, dev)

trainLoader, testLoader = data.BucketIterator.splits((train, dev), shuffle=True, batch_size=64,
                                                     sort_key=lambda x: len(x.text), sort_within_batch=True)

In [173]:
lstm1 = tnn.LSTM(50, 200, batch_first=True)

conv = tnn.Conv1d(1, 24, 5, padding=5)
pool = tnn.MaxPool1d(4)
dense = tnn.Linear(24*51,400)
lstm2 = tnn.LSTM(400, 256,batch_first=True)

In [175]:
shapes = []
for i, batch in enumerate(trainLoader, 0):
                inputs, length, labels = textField.vocab.vectors[batch.text[0]].to(device), batch.text[1].to(
                device), batch.label.type(torch.FloatTensor).to(device)
                labels -= 1
                o  = lstm(inputs)[1][0].permute(1,0,2)   
                o  = conv(o)
                o = pool(o)
                o = dense(o.view(-1,24*51))
                o = lstm2(o.view(-1,1,400))[1][0]
                shapes.append(o.shape)


In [113]:
o  = lstm(inputs)[1][0].permute(1,0,2)

In [118]:
o.shape

torch.Size([64, 24, 49])

In [176]:
shapes

[torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 torch.Size([1, 64, 256]),
 

# 构建神经网络

In [282]:
# Class for creating the neural network.
class Network(tnn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        
        self.optimizer = None
        self.criterion = None     

        self.lstm1 = tnn.LSTM(50, 200, batch_first=True)
        self.conv1 = tnn.Conv1d(1, 24, 5, padding=5)
        self.relu = tnn.ReLU()
        self.maxpool = tnn.MaxPool1d(4)
        # TODO -> 先Dropout还是先normalize ？
        self.dropout = tnn.Dropout()
        
        self.dense1 = tnn.Linear(24*51, 400)
        self.norm1 = tnn.BatchNorm1d(400)
        

        # Relu
        self.dense2 = tnn.Linear(400, 256)
        # dropout
        self.dense3 = tnn.Linear(256, 64)
        # Relu
        self.dense4 = tnn.Linear(64, 1)
        
        self.to(self.device)
        print(self.device)    
        
        
    def compile(self, optimizer, criterion):
        self.optimizer = optimizer
        self.criterion = criterion

    def fit(self, trainset, EPOCHS):
        for epoch in range(EPOCHS):
            running_loss = 0.0
            for i, batch in enumerate(trainset, 0):
                inputs, length, labels = textField.vocab.vectors[batch.text[0]].to(device), batch.text[1].to(
                device), batch.label.type(torch.FloatTensor).to(device)
                labels -= 1
                self.optimizer.zero_grad()
                #tnn.functional.batch_norm(inputs,0.5,0.5,training=True)
                
                predict = self(inputs.to(self.device),length)
                loss = criterion(predict.view(-1), labels.to(self.device))
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()
                
                if i % 32 == 31:
                    print("Epoch: %2d, Batch: %4d, Loss: %.3f" % 
                              (epoch + 1, i + 1, running_loss / 32))
                    running_loss = 0
        print('trainning completed!')
                
        
    def conv(self, features):
        x = self.lstm1(features)[1][0].permute(1,0,2)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.dropout(x)
        
    
        return x

    def forward(self, input, length):
        x = self.conv(input)
        x = self.dense1(x.view(-1,24*51))
        x = self.norm1(x)
        x = self.relu(x)

        x = self.dense2(x)
        x = self.dropout(x)
        x = torch.tanh(x)
        x = self.dense3(x)
        x = self.relu(x)
        x = self.dense4(x)
        
        return tnn.functional.logsigmoid(x, dim=1)

        

In [279]:
net = Network()
print(net)

cuda:0
Network(
  (lstm1): LSTM(50, 200, batch_first=True)
  (conv1): Conv1d(1, 24, kernel_size=(5,), stride=(1,), padding=(5,))
  (relu): ReLU()
  (maxpool): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (dense1): Linear(in_features=1224, out_features=400, bias=True)
  (norm1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense2): Linear(in_features=400, out_features=256, bias=True)
  (dense3): Linear(in_features=256, out_features=64, bias=True)
  (dense4): Linear(in_features=64, out_features=1, bias=True)
)


In [289]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
criterion = tnn.MSELoss()
net.compile(optimizer, criterion)

In [290]:
net.fit(trainLoader,3)

/home/reco/Applications/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch:  1, Batch:   32, Loss: 0.498
Epoch:  1, Batch:   64, Loss: 0.511
Epoch:  1, Batch:   96, Loss: 0.488
Epoch:  1, Batch:  128, Loss: 0.494
Epoch:  1, Batch:  160, Loss: 0.479
Epoch:  1, Batch:  192, Loss: 0.506
Epoch:  1, Batch:  224, Loss: 0.494
Epoch:  1, Batch:  256, Loss: 0.518
Epoch:  1, Batch:  288, Loss: 0.490
Epoch:  1, Batch:  320, Loss: 0.502
Epoch:  1, Batch:  352, Loss: 0.503


/home/reco/Applications/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([40])) that is different to the input size (torch.Size([40, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch:  1, Batch:  384, Loss: 0.529
Epoch:  2, Batch:   32, Loss: 0.490
Epoch:  2, Batch:   64, Loss: 0.511
Epoch:  2, Batch:   96, Loss: 0.519
Epoch:  2, Batch:  128, Loss: 0.480
Epoch:  2, Batch:  160, Loss: 0.510
Epoch:  2, Batch:  192, Loss: 0.517
Epoch:  2, Batch:  224, Loss: 0.486
Epoch:  2, Batch:  256, Loss: 0.510
Epoch:  2, Batch:  288, Loss: 0.485
Epoch:  2, Batch:  320, Loss: 0.480
Epoch:  2, Batch:  352, Loss: 0.515
Epoch:  2, Batch:  384, Loss: 0.498
Epoch:  3, Batch:   32, Loss: 0.511
Epoch:  3, Batch:   64, Loss: 0.497
Epoch:  3, Batch:   96, Loss: 0.503
Epoch:  3, Batch:  128, Loss: 0.491
Epoch:  3, Batch:  160, Loss: 0.504
Epoch:  3, Batch:  192, Loss: 0.500
Epoch:  3, Batch:  224, Loss: 0.506
Epoch:  3, Batch:  256, Loss: 0.488
Epoch:  3, Batch:  288, Loss: 0.502
Epoch:  3, Batch:  320, Loss: 0.490
Epoch:  3, Batch:  352, Loss: 0.481
Epoch:  3, Batch:  384, Loss: 0.515
trainning completed!


In [ ]:
class PreProcessing():
    def pre(x):
        """Called after tokenization"""
        return x

    def post(batch, vocab):
        """Called after numericalization but prior to vectorization"""
        return batch, vocab

    text_field = data.Field(lower=True, include_lengths=True, batch_first=True, preprocessing=pre, postprocessing=post)

In [ ]:
def lossFunc():
    """
    Define a loss function appropriate for the above networks that will
    add a sigmoid to the output and calculate the binary cross-entropy.
    """

In [ ]:
def main():
    # Use a GPU if available, as it should be faster.
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Using device: " + str(device))

    # Load the training dataset, and create a data loader to generate a batch.
    textField = PreProcessing.text_field
    labelField = data.Field(sequential=False)

    train, dev = IMDB.splits(textField, labelField, train="train", validation="dev")

    textField.build_vocab(train, dev, vectors=GloVe(name="6B", dim=50))
    labelField.build_vocab(train, dev)

    trainLoader, testLoader = data.BucketIterator.splits((train, dev), shuffle=True, batch_size=64,
                                                         sort_key=lambda x: len(x.text), sort_within_batch=True)

    net = Network().to(device)
    criterion =lossFunc()
    optimiser = topti.Adam(net.parameters(), lr=0.001)  # Minimise the loss using the Adam algorithm.

    for epoch in range(10):
        running_loss = 0

        for i, batch in enumerate(trainLoader):
            # Get a batch and potentially send it to GPU memory.
            inputs, length, labels = textField.vocab.vectors[batch.text[0]].to(device), batch.text[1].to(
                device), batch.label.type(torch.FloatTensor).to(device)

            labels -= 1

            # PyTorch calculates gradients by accumulating contributions to them (useful for
            # RNNs).  Hence we must manually set them to zero before calculating them.
            optimiser.zero_grad()

            # Forward pass through the network.
            output = net(inputs, length)

            loss = criterion(output, labels)

            # Calculate gradients.
            loss.backward()

            # Minimise the loss according to the gradient.
            optimiser.step()

            running_loss += loss.item()

            if i % 32 == 31:
                print("Epoch: %2d, Batch: %4d, Loss: %.3f" % (epoch + 1, i + 1, running_loss / 32))
                running_loss = 0

    num_correct = 0

    # Save mode
    torch.save(net.state_dict(), "./model.pth")
    print("Saved model")

    # Evaluate network on the test dataset.  We aren't calculating gradients, so disable autograd to speed up
    # computations and reduce memory usage.
    with torch.no_grad():
        for batch in testLoader:
            # Get a batch and potentially send it to GPU memory.
            inputs, length, labels = textField.vocab.vectors[batch.text[0]].to(device), batch.text[1].to(
                device), batch.label.type(torch.FloatTensor).to(device)

            labels -= 1

            # Get predictions
            outputs = torch.sigmoid(net(inputs, length))
            predicted = torch.round(outputs)

            num_correct += torch.sum(labels == predicted).item()

    accuracy = 100 * num_correct / len(dev)

    print(f"Classification accuracy: {accuracy}")

if __name__ == '__main__':
    main()